# 📊 AutoCron Dashboard & Visual Monitoring Demo

Welcome to the **Dashboard & Visual Monitoring** demo! This notebook covers:

✨ **What You'll Learn:**
- 📈 Basic dashboard usage with `show_dashboard()`
- 🔍 Task-specific analytics with `show_task()`
- ⚡ Live monitoring with refresh rates
- 🎨 Rich terminal UI features
- 📊 Viewing task execution history
- 📉 Performance metrics and statistics
- 🔄 Real-time task monitoring
- 🎯 Dashboard in different scenarios
- 🛠️ Dashboard customization options

**Prerequisites:** Notebooks 01-05  
**Next:** 07_notifications.ipynb

## 🔧 Setup

In [ ]:
from autocron import AutoCron
from autocron.interface.dashboard import show_dashboard, show_task
import time
from datetime import datetime

scheduler = AutoCron()

print("✅ Setup complete!")
print("📊 Dashboard modules loaded")

## 1️⃣ Basic Dashboard - Overview of All Tasks

The `show_dashboard()` function displays a beautiful overview of all scheduled tasks.

In [ ]:
# Schedule some sample tasks
@scheduler.interval(seconds=10)
def system_health_check():
    """Check system health metrics"""
    print("🏥 Running health check...")
    return {"status": "healthy", "timestamp": datetime.now()}

@scheduler.cron("0 9 * * *")
def daily_backup():
    """Perform daily backup at 9 AM"""
    print("💾 Backing up data...")
    return {"files_backed_up": 150}

@scheduler.interval(minutes=5)
def sync_data():
    """Sync data every 5 minutes"""
    print("🔄 Syncing data...")
    return {"records_synced": 42}

# View the dashboard
print("📊 Displaying Dashboard:\n")
show_dashboard(scheduler)

print("\n✅ Dashboard shows:")
print("  • Task names and descriptions")
print("  • Schedule patterns")
print("  • Next run times")
print("  • Task status")

## 2️⃣ Task-Specific Analytics with show_task()

Get detailed information about a specific task including execution history and statistics.

In [ ]:
# Run a task multiple times to build history
print("🏃 Running 'system_health_check' a few times...\n")

for _ in range(3):
    system_health_check()
    time.sleep(0.5)

# Show detailed task analytics
print("\n📈 Task Analytics for 'system_health_check':\n")
show_task(scheduler, "system_health_check")

print("\n✅ Task details show:")
print("  • Execution count")
print("  • Success/failure rate")
print("  • Average execution time")
print("  • Last execution timestamp")
print("  • Next scheduled run")

## 3️⃣ Dashboard with Task Statistics

View comprehensive statistics across all tasks including success rates and performance metrics.

In [ ]:
# Get scheduler statistics
stats = scheduler.get_stats()

print("📊 Scheduler Statistics:")
print(f"  • Total tasks: {stats.get('total_tasks', 0)}")
print(f"  • Active tasks: {stats.get('active_tasks', 0)}")
print(f"  • Total executions: {stats.get('total_executions', 0)}")
print(f"  • Successful: {stats.get('successful_executions', 0)}")
print(f"  • Failed: {stats.get('failed_executions', 0)}")
print(f"  • Success rate: {stats.get('success_rate', 0):.1f}%")

# Dashboard with detailed view
print("\n📈 Detailed Dashboard View:\n")
show_dashboard(scheduler, detailed=True)

print("\n💡 Use cases:")
print("  • Monitor system health")
print("  • Track task performance")
print("  • Identify bottlenecks")
print("  • Debug scheduling issues")

## 4️⃣ Real-Time Monitoring Example

Monitor tasks as they execute with live execution tracking.

In [ ]:
import threading

# Create a monitoring task
@scheduler.interval(seconds=2)
def monitored_task():
    """Task that runs every 2 seconds for monitoring"""
    print(f"⚡ Task executed at {datetime.now().strftime('%H:%M:%S')}")
    time.sleep(0.1)  # Simulate work
    return {"status": "completed"}

# Start scheduler in background
print("🚀 Starting scheduler with monitoring...\n")
thread = threading.Thread(target=scheduler.start, daemon=True)
thread.start()

# Monitor for a few seconds
print("👀 Monitoring task executions:")
for i in range(5):
    time.sleep(1)
    if i % 2 == 0:
        # Show dashboard every 2 seconds
        print(f"\n📊 Dashboard update {i//2 + 1}:")
        show_dashboard(scheduler)

scheduler.stop()
print("\n✅ Monitoring session complete!")

print("\n💡 Real-time monitoring helps:")
print("  • Track task execution in production")
print("  • Detect delays or failures immediately")
print("  • Monitor resource usage")
print("  • Debug scheduling issues live")

## 5️⃣ Dashboard Scenarios - Success, Failures, and Mixed States

See how the dashboard displays different task states.

In [ ]:
# Create a new scheduler for scenarios
scenario_scheduler = AutoCron()

# Successful task
@scenario_scheduler.interval(seconds=30)
def successful_task():
    """Always succeeds"""
    return {"status": "success"}

# Failing task
@scenario_scheduler.interval(seconds=45)
def failing_task():
    """Simulates a failing task"""
    raise Exception("Database connection failed")

# Mixed results task
import random
@scenario_scheduler.interval(seconds=60)
def mixed_task():
    """Sometimes succeeds, sometimes fails"""
    if random.random() > 0.5:
        return {"status": "success"}
    else:
        raise Exception("Random failure occurred")

# Simulate some executions
import contextlib
print("🎭 Simulating different task scenarios...\n")

# Run successful task
with contextlib.suppress(Exception):
    successful_task()
    print("✅ Successful task completed")
# Run failing task
try:
    failing_task()
except Exception as e:
    print(f"❌ Failing task error: {e}")

# Run mixed task multiple times
for i in range(3):
    try:
        mixed_task()
        print(f"✅ Mixed task attempt {i+1}: Success")
    except Exception as e:
        print(f"❌ Mixed task attempt {i+1}: Failed - {e}")

# Show dashboard with all states
print("\n📊 Dashboard showing mixed task states:\n")
show_dashboard(scenario_scheduler)

print("\n💡 Dashboard color coding:")
print("  • 🟢 Green: Healthy tasks")
print("  • 🟡 Yellow: Warning state")
print("  • 🔴 Red: Failed tasks")
print("  • ⚪ Gray: Not yet executed")

## 6️⃣ Rich Terminal UI Features

AutoCron uses the `rich` library for beautiful terminal output with colors, tables, and progress bars.

In [ ]:
from rich.console import Console
from rich.table import Table
from rich.progress import Progress

console = Console()

# Create a rich table for task overview
table = Table(title="📋 AutoCron Tasks Overview", show_header=True, header_style="bold magenta")
table.add_column("Task Name", style="cyan")
table.add_column("Schedule", style="green")
table.add_column("Status", style="yellow")
table.add_column("Last Run", style="blue")

# Add task rows
table.add_row("system_health_check", "Every 10s", "✅ Active", "2 min ago")
table.add_row("daily_backup", "0 9 * * *", "⏰ Scheduled", "Never")
table.add_row("sync_data", "Every 5min", "✅ Active", "30s ago")

console.print(table)

# Progress bar example
print("\n🔄 Simulating task execution progress:\n")
with Progress() as progress:
    task1 = progress.add_task("[cyan]Processing data...", total=100)
    task2 = progress.add_task("[green]Uploading files...", total=100)
    task3 = progress.add_task("[yellow]Cleaning up...", total=100)
    
    while not progress.finished:
        progress.update(task1, advance=5)
        progress.update(task2, advance=3)
        progress.update(task3, advance=7)
        time.sleep(0.1)

print("\n✨ Rich UI features:")
print("  • Color-coded output")
print("  • Beautiful tables")
print("  • Progress bars")
print("  • Status indicators")
print("  • Emoji support 🎉")

## 7️⃣ Production Dashboard Monitoring

Complete example of using the dashboard in a production environment.

In [ ]:
# Production monitoring setup
prod_scheduler = AutoCron()

# Critical production tasks
@prod_scheduler.interval(minutes=1, metadata={"priority": "critical"})
def monitor_api_health():
    """Monitor API endpoint health"""
    # Simulate API check
    return {"status": "healthy", "response_time_ms": 45}

@prod_scheduler.interval(minutes=5, metadata={"priority": "high"})
def check_database_connections():
    """Verify database connection pool"""
    return {"active_connections": 12, "max_connections": 100}

@prod_scheduler.interval(minutes=15, metadata={"priority": "medium"})
def cleanup_temp_files():
    """Clean up temporary files"""
    return {"files_deleted": 23, "space_freed_mb": 150}

@prod_scheduler.cron("0 */6 * * *", metadata={"priority": "low"})
def generate_analytics_report():
    """Generate 6-hour analytics report"""
    return {"report_id": "REPT-2024-001", "records_processed": 50000}

# Run some tasks
print("🏭 Production System - Running health checks...\n")
monitor_api_health()
check_database_connections()
cleanup_temp_files()

# Show production dashboard
print("\n📊 Production Dashboard:\n")
show_dashboard(prod_scheduler)

# Show critical task details
print("\n🔍 Critical Task Details:\n")
show_task(prod_scheduler, "monitor_api_health")

# Get comprehensive stats
print("\n📈 System Health Summary:")
stats = prod_scheduler.get_stats()
print(f"  • Total tasks monitored: {len(prod_scheduler.tasks)}")
print(f"  • Success rate: {stats.get('success_rate', 100):.1f}%")
print("   • System uptime: 99.9%")
print(f"  • Next critical task: {prod_scheduler.get_next_run()}")

print("\n💼 Production best practices:")
print("  • Monitor critical tasks every 1-5 minutes")
print("  • Use priority metadata for task categorization")
print("  • Set up automated health checks")
print("  • Review dashboard daily")
print("  • Track success rates and trends")
print("  • Use show_task() for detailed diagnostics")

## 🧹 Cleanup

In [ ]:
# Stop all schedulers
import contextlib
for sched in [scheduler, scenario_scheduler, prod_scheduler]:
    with contextlib.suppress(Exception):
        sched.stop()
print("✅ All schedulers stopped and cleaned up!")

## 📚 Summary

### What You Learned:
✅ **Basic Dashboard** - `show_dashboard()` for overview of all tasks  
✅ **Task Analytics** - `show_task()` for detailed task insights  
✅ **Statistics** - `get_stats()` for comprehensive metrics  
✅ **Real-Time Monitoring** - Live task execution tracking  
✅ **Task States** - Success, failure, and warning indicators  
✅ **Rich UI** - Beautiful terminal output with colors and tables  
✅ **Production Monitoring** - Enterprise-grade dashboard usage

### Key Dashboard Functions:
```python
show_dashboard(scheduler)              # Show all tasks
show_dashboard(scheduler, detailed=True)  # Detailed view
show_task(scheduler, "task_name")      # Task-specific details
scheduler.get_stats()                  # Get statistics dictionary
```

### Production Use Cases:
- 🏥 **Health Monitoring** - Track system health checks
- 📊 **Performance Analytics** - Monitor task execution times
- 🔍 **Debugging** - Identify failing or slow tasks
- 📈 **Capacity Planning** - Analyze task load and trends
- 🚨 **Alerting** - Quick identification of issues

### Best Practices:
1. ✅ Check dashboard regularly in production
2. ✅ Use `show_task()` for detailed diagnostics
3. ✅ Monitor success rates and execution times
4. ✅ Set up priority metadata for task categorization
5. ✅ Review failed tasks immediately
6. ✅ Track trends over time with `get_stats()`

### Next Steps:
- 📧 **07_notifications.ipynb** - Set up email and desktop alerts
- 🖥️ **08_cli_and_logging.ipynb** - Use CLI commands and logging

**Happy Monitoring! 📊✨**